In [ ]:
%cd /content
!wget https://github.com/karoldvl/ESC-50/archive/master.zip

In [ ]:
!mkdir -p /content/speech_commands
%cd /content/speech_commands
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
!tar -xzf speech_commands_v0.02.tar.gz

In [ ]:
!rm -f /content/esc_audio_16k
!mkdir -p /content/esc_audio_16k
%cd /content/ESC-50-master/audio
!for i in *.wav; do name=`echo "${i%.*}"` ; ffmpeg -y -i "${name}.wav" -ar 16000 -ac 1 "/content/esc_audio_16k/${name}.wav"; done

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Conv2D, Dense, GlobalAveragePooling2D, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model, Sequence

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import matplotlib
import matplotlib.pyplot as plt
from shutil import copyfile

In [ ]:
%cd /content
!git clone https://github.com/train255/kambria-challenge
%cd /content/kambria-challenge/generateMFCC/
!javac -source 1.7 -target 1.7 -d bin -cp lib/jlibrosa-1.1.8-SNAPSHOT-jar-with-dependencies.jar src/com/example/Main.java

In [ ]:
!pip install pydub

### Load data

In [ ]:
meta_df = pd.read_csv("/content/ESC-50-master/meta/esc50.csv")
meta_df["audio_path"] = meta_df["filename"].apply(lambda x: "/content/esc_audio_16k/"+x)
meta_df["uuid"] = meta_df["filename"].apply(lambda x: x.split(".wav")[0])
meta_df

In [ ]:
speech_validation_df = pd.read_csv("/content/speech_commands/validation_list.txt", header=None).reset_index(drop=True)
speech_validation_df = speech_validation_df.rename(columns = {0: 'path'})
speech_validation_df

In [ ]:
speech_testing_df = pd.read_csv("/content/speech_commands/testing_list.txt", header=None).reset_index(drop=True)
speech_testing_df = speech_testing_df.rename(columns = {0: 'path'})
speech_testing_df

In [ ]:
speech_df = pd.concat([speech_validation_df, speech_testing_df])
speech_df = speech_df.sort_values("path").reset_index(drop=True)
speech_df["category"] = speech_df["path"].apply(lambda x: os.path.dirname(x))
speech_df["uuid"] = speech_df["path"].apply(lambda x: os.path.basename(x).split(".wav")[0])
speech_df["audio_path"] = speech_df["path"].apply(lambda x: "/content/speech_commands/"+x)
speech_df

In [ ]:
train_meta_df = pd.read_csv("/content/aicv115m_final_public_train/public_train_metadata.csv")
train_audio_dir = "/content/final_public_train/"
train_meta_df["audio_path"] = train_meta_df["uuid"].apply(lambda x: train_audio_dir + x + ".wav")
train_meta_df.drop(train_meta_df[train_meta_df['uuid'] == "23ccaa28-8cb8-43e4-9e59-112fa4dc6559"].index, inplace = True)
idx_lst = [169, 1063, 2064, 2297, 2512, 2723, 2832, 3143, 3600, 3774, 3820, 4191, 4378]
df_covid = train_meta_df.iloc[idx_lst][["uuid","assessment_result", "audio_path"]].reset_index(drop=True)
train_nonote_df = train_meta_df[train_meta_df.audio_noise_note.isnull()].reset_index(drop=True)
train_nonote_df = train_nonote_df[["uuid","assessment_result", "audio_path"]]
train_data = pd.concat([train_nonote_df, df_covid]).reset_index(drop=True)
train_data = train_data[["uuid","assessment_result", "audio_path"]]

In [ ]:
# Get duration and sample rate
import scipy
from scipy.io import wavfile

def getRateAndDuration(_df):
    sample_rate_lst = []
    seconds_lst = []
    for index, row in _df.iterrows():
        sample_rate, data = wavfile.read(row["audio_path"])
        sample_rate_lst.append(sample_rate)
        len_data = len(data)  # holds length of the numpy array
        t = len_data / sample_rate  # returns duration but in floats
        seconds_lst.append(t)
    _df["sample_rate"] = sample_rate_lst
    _df["seconds"] = seconds_lst
    return _df

In [ ]:
train_data = getRateAndDuration(train_data)
cough_aicovid_df = train_data[(train_data["seconds"] <= 6.13)].reset_index(drop=True)
cough_aicovid_df["assessment_result"] = [1] * len(cough_aicovid_df)

In [ ]:
background_noises = ["doing_the_dishes", "dude_miaowing", "exercise_bike",
                             "pink_noise", "running_tap", "white_noise"]
bg_data = []
for i in background_noises:
    uuid = i
    aug_pth = "/content/speech_commands/_background_noise_/" + uuid + ".wav"
    bg_data.append([uuid, aug_pth])

bg_noise_df = pd.DataFrame(bg_data, columns = ['uuid', 'audio_path'])
bg_noise_df

#### Positive Data

In [ ]:
from pydub import AudioSegment
import random
import os

def mixAudio(cough_path, noise_path, dst_dir):
    cough_name = os.path.basename(cough_path)
    noise_name = os.path.basename(noise_path)
    uuid = cough_name.split(".wav")[0]+noise_name.split(".wav")[0]
    new_name = uuid+".wav"
    sound1 = AudioSegment.from_file(cough_path, format="wav")
    sound2 = AudioSegment.from_file(noise_path, format="wav")

    # sound1 6 dB louder
    sound2_aug = sound2 - random.randint(8, 12)

    # sound1, with sound2 appended (use louder instead of sound1 to append the louder version)
    combined = sound1.overlay(sound2_aug)
    pth = dst_dir+new_name
    # simple export
    combined.export(pth, format="wav")
    return pth, uuid

def mixBackground(_df, aug_df, label, dst_dir):
    mix_data = []
    for index, row in _df.iterrows():
        for index2, row_aug in aug_df.iterrows():
            aug_pth, uuid = mixAudio(row["audio_path"], row_aug["audio_path"], dst_dir)
            mix_data.append([uuid, aug_pth, label])
    return pd.DataFrame(mix_data, columns = ['uuid', 'audio_path', 'assessment_result'])

In [ ]:
def addLabel(_df, label):
    _df["assessment_result"] = [label] * len(_df)
    return _df[["uuid", "audio_path", "assessment_result"]].reset_index(drop=True)

In [ ]:
cough_esc_df_train = meta_df[(meta_df["fold"] < 4) & (meta_df["category"] == "coughing")].reset_index(drop=True)
cough_esc_df_valid = meta_df[(meta_df["fold"] == 4) & (meta_df["category"] == "coughing")].reset_index(drop=True)
cough_esc_df_test = meta_df[(meta_df["fold"] == 5) & (meta_df["category"] == "coughing")].reset_index(drop=True)

In [ ]:
wind_bg_df = meta_df[(meta_df["category"]=="wind") & (meta_df["src_file"] <= 47709)]
rain_bg_df = meta_df[(meta_df["category"]=="rain") & (meta_df["src_file"] <= 21189)]
clock_tick_bg_df = meta_df[(meta_df["category"]=="clock_tick") & (meta_df["src_file"] <= 21935)]
footsteps_bg_df = meta_df[(meta_df["category"]=="footsteps") & (meta_df["src_file"] <= 51149)]
esc_bg_df = pd.concat([wind_bg_df, rain_bg_df, clock_tick_bg_df, footsteps_bg_df])
esc_bg_idx = esc_bg_df.index.tolist()
cough_esc_data_background = esc_bg_df.reset_index(drop=True)

In [ ]:
!rm -rf "/content/cough_aug/"
!mkdir -p "/content/cough_aug/"
cough_mix_esc_df = mixBackground(cough_esc_df_train, cough_esc_data_background, 1, "/content/cough_aug/")

In [ ]:
go_bg_df = speech_df[(speech_df["category"]=="go") & (speech_df.index <= 5624)]
up_bg_df = speech_df[(speech_df["category"]=="up") & (speech_df.index <= 17891)]
down_bg_df = speech_df[(speech_df["category"]=="down") & (speech_df.index <= 1897)]
right_bg_df = speech_df[(speech_df["category"]=="right") & (speech_df.index <= 12508)]
left_bg_df = speech_df[(speech_df["category"]=="left") & (speech_df.index <= 7495)]
yes_bg_df = speech_df[(speech_df["category"]=="yes") & (speech_df.index <= 19369)]
no_bg_df = speech_df[(speech_df["category"]=="no") & (speech_df.index <= 9413)]
speech_bg_df = pd.concat([go_bg_df, up_bg_df, down_bg_df, right_bg_df, left_bg_df, yes_bg_df, no_bg_df])
speech_bg_idx = speech_bg_df.index.tolist()
cough_speech_data_background = speech_bg_df.reset_index(drop=True)

In [ ]:
!rm -rf "/content/cough_aug2/"
!mkdir -p "/content/cough_aug2/"
cough_mix_speech_df = mixBackground(cough_esc_df_train, cough_speech_data_background, 1, "/content/cough_aug2/")

In [ ]:
cough_aicovid_df_train, cough_aicovid_df_valid, cough_aicovid_df_test = np.split(cough_aicovid_df.sample(frac=1, random_state=42), [int(.8*len(cough_aicovid_df)), int(.9*len(cough_aicovid_df))])
cough_aicovid_df_train = cough_aicovid_df_train[["uuid", "audio_path", "assessment_result"]].reset_index(drop=True)
cough_aicovid_df_valid = cough_aicovid_df_valid[["uuid", "audio_path", "assessment_result"]].reset_index(drop=True)
cough_aicovid_df_test = cough_aicovid_df_test[["uuid", "audio_path", "assessment_result"]].reset_index(drop=True)

In [ ]:
!rm -rf "/content/cough_aug3/"
!mkdir -p "/content/cough_aug3/"
cough_ai_mix_esc_df = mixBackground(cough_aicovid_df_train, cough_esc_data_background, 1, "/content/cough_aug3/")

In [ ]:
!rm -rf "/content/cough_aug4/"
!mkdir -p "/content/cough_aug4/"
cough_ai_mix_speech_df = mixBackground(cough_aicovid_df_train, cough_speech_data_background, 1, "/content/cough_aug4/")

In [ ]:
!rm -rf "/content/cough_aug5/"
!mkdir -p "/content/cough_aug5/"
cough_mix_bg_noise_df = mixBackground(cough_esc_df_train, bg_noise_df, 1, "/content/cough_aug5/")

In [ ]:
!rm -rf "/content/cough_aug6/"
!mkdir -p "/content/cough_aug6/"
cough_ai_mix_bg_noise_df = mixBackground(cough_aicovid_df_train, bg_noise_df, 1, "/content/cough_aug6/")

In [ ]:
cough_esc_df_train = addLabel(cough_esc_df_train, 1)
cough_esc_df_valid = addLabel(cough_esc_df_valid, 1)
cough_esc_df_test = addLabel(cough_esc_df_test, 1)

In [ ]:
positive_df_train = pd.concat([cough_esc_df_train, cough_mix_esc_df, cough_mix_speech_df,
                               cough_aicovid_df_train, cough_ai_mix_esc_df, cough_ai_mix_speech_df,
                               cough_mix_bg_noise_df, cough_ai_mix_bg_noise_df]).reset_index(drop=True)

positive_df_valid = pd.concat([cough_esc_df_valid, cough_aicovid_df_valid]).reset_index(drop=True)
positive_df_test = pd.concat([cough_esc_df_test, cough_aicovid_df_test]).reset_index(drop=True)

In [ ]:
print(len(positive_df_train))
print(len(positive_df_valid))
print(len(positive_df_test))

#### Negative Data

In [ ]:
no_cough_esc_df = meta_df[(~meta_df.index.isin(esc_bg_idx)) & (meta_df["category"] != "coughing")].reset_index(drop=True)
no_cough_esc_df_train = no_cough_esc_df[no_cough_esc_df["fold"] < 4].reset_index(drop=True)
no_cough_esc_df_valid = no_cough_esc_df[no_cough_esc_df["fold"] == 4].reset_index(drop=True)
no_cough_esc_df_test = no_cough_esc_df[no_cough_esc_df["fold"] == 5].reset_index(drop=True)

In [ ]:
no_cough_speech_df = speech_df[~speech_df.index.isin(speech_bg_idx)].reset_index(drop=True)
X_train, speech_other_df_test = train_test_split(no_cough_speech_df, test_size=0.1, random_state=42)
speech_other_df_train, speech_other_df_valid = train_test_split(X_train, test_size=0.11, random_state=42)
speech_other_df_train = speech_other_df_train.reset_index(drop=True)
speech_other_df_valid = speech_other_df_valid.reset_index(drop=True)
speech_other_df_test = speech_other_df_test.reset_index(drop=True)

In [ ]:
!rm -rf "/content/no_cough_aug1/"
!mkdir -p "/content/no_cough_aug1/"
no_cough_esc_mix_bg_noise_df = mixBackground(no_cough_esc_df_train, bg_noise_df, 0, "/content/no_cough_aug1/")

In [ ]:
def mixRandomBackground(_df, aug_df, label, dst_dir, n):
    mix_data = []
    for index, row in _df.iterrows():
        aug_rand = aug_df.sample(n=n).reset_index(drop=True)
        for index2, row_aug in aug_rand.iterrows():
            aug_pth, uuid = mixAudio(row["audio_path"], row_aug["audio_path"], dst_dir)
            mix_data.append([uuid, aug_pth, label])
    return pd.DataFrame(mix_data, columns = ['uuid', 'audio_path', 'assessment_result'])

In [ ]:
!rm -rf "/content/no_cough_aug2/"
!mkdir -p "/content/no_cough_aug2/"
speech_mix_bg_noise_df = mixRandomBackground(speech_other_df_train, bg_noise_df, 0, "/content/no_cough_aug2/", 1)

In [ ]:
no_cough_esc_df_train = addLabel(no_cough_esc_df_train, 0)
speech_other_df_train = addLabel(speech_other_df_train, 0)
no_cough_esc_df_valid = addLabel(no_cough_esc_df_valid, 0)
no_cough_esc_df_test = addLabel(no_cough_esc_df_test, 0)
speech_other_df_valid = addLabel(speech_other_df_valid, 0)
speech_other_df_test = addLabel(speech_other_df_test, 0)

In [ ]:
negative_df_train = pd.concat([no_cough_esc_df_train, speech_other_df_train,
                               no_cough_esc_mix_bg_noise_df, speech_mix_bg_noise_df]).reset_index(drop=True)

In [ ]:
negative_df_valid = pd.concat([no_cough_esc_df_valid, speech_other_df_valid]).reset_index(drop=True)
negative_df_test = pd.concat([no_cough_esc_df_test, speech_other_df_test]).reset_index(drop=True)

In [ ]:
print(len(negative_df_train))
print(len(negative_df_valid))
print(len(negative_df_test))

### Create features

In [ ]:
num_rows = 120
num_columns = 192
num_channels = 1

n_fft = 4096
hop_length = 512
n_mels = 512

from pathlib import Path
import re

def add_pad_len(x):
    if x.shape[1] <= num_columns:
        pad_width = num_columns - x.shape[1]
        x = np.pad(x, pad_width=((0,0),(0,pad_width)), mode='constant')
    return x

def readFeatures(uuid, features_dir):
    txt_pth = features_dir + uuid + ".txt"
    mfcc_txt = Path(txt_pth).read_text()
    cols = []
    for cols_txt in mfcc_txt.split("n"):
        if cols_txt != "":
            rows_str = cols_txt.split(",")
            rows = []
            for row in rows_str:
                # if row != "" and isFloatFormat(row):
                if row != "":
                    rows.append(float(row))
            cols.append(rows)
    features = np.array(cols, dtype=np.float)
    if (features.shape[0] > 192 or features.shape[1] > 192):
        print(features.shape)
        print(uuid)
    features = add_pad_len(features)
    return features

In [ ]:
!rm -rf /content/cough_data
!mkdir -p /content/cough_data
for index, row in positive_df_train.iterrows():
    src = row["audio_path"]
    dst = "/content/cough_data/"+row["uuid"]+".wav"
    copyfile(src, dst)
positive_df_train['audio_path'] = positive_df_train['audio_path'].apply(lambda x: "/content/cough_data/"+os.path.basename(x))

In [ ]:
!rm -rf /content/cough_features
!mkdir -p /content/cough_features
%cd /content/kambria-challenge/generateMFCC/
!java -cp lib/jlibrosa-1.1.8-SNAPSHOT-jar-with-dependencies.jar:bin com.example.Main /content/cough_data /content/cough_features 16000 120

In [ ]:
positive_df_train["feature"] = positive_df_train["uuid"].apply(readFeatures, features_dir="/content/cough_features/")

In [ ]:
!rm -rf /content/no_cough_data
!mkdir -p /content/no_cough_data
for index, row in negative_df_train.iterrows():
    src = row["audio_path"]
    dst = "/content/no_cough_data/"+row["uuid"]+".wav"
    copyfile(src, dst)
negative_df_train['audio_path'] = negative_df_train['audio_path'].apply(lambda x: "/content/no_cough_data/"+os.path.basename(x))

In [ ]:
!rm -rf /content/no_cough_features
!mkdir -p /content/no_cough_features
%cd /content/kambria-challenge/generateMFCC/
!java -cp lib/jlibrosa-1.1.8-SNAPSHOT-jar-with-dependencies.jar:bin com.example.Main /content/no_cough_data /content/no_cough_features 16000 120

In [ ]:
negative_df_train["feature"] = negative_df_train["uuid"].apply(readFeatures, features_dir="/content/no_cough_features/")

In [ ]:
%cd /content
!unzip -q /content/drive/MyDrive/ML/cough_recognition/cough_data.zip -d ./
!unzip -q /content/drive/MyDrive/ML/cough_recognition/cough_features.zip -d ./
!unzip -q /content/drive/MyDrive/ML/cough_recognition/no_cough_data.zip -d ./
!unzip -q /content/drive/MyDrive/ML/cough_recognition/no_cough_features.zip -d ./

In [ ]:
final_aug_data = pd.concat([positive_df_train, negative_df_train])

In [ ]:
!rm -rf /content/cough_data_valid
!mkdir -p /content/cough_data_valid
for index, row in positive_df_valid.iterrows():
    src = row["audio_path"]
    dst = "/content/cough_data_valid/"+row["uuid"]+".wav"
    copyfile(src, dst)
positive_df_valid['audio_path'] = positive_df_valid['audio_path'].apply(lambda x: "/content/cough_data_valid/"+os.path.basename(x))

In [ ]:
!rm -rf /content/cough_valid_features
!mkdir -p /content/cough_valid_features
%cd /content/kambria-challenge/generateMFCC/
!java -cp lib/jlibrosa-1.1.8-SNAPSHOT-jar-with-dependencies.jar:bin com.example.Main /content/cough_data_valid /content/cough_valid_features 16000 120

In [ ]:
positive_df_valid["feature"] = positive_df_valid["uuid"].apply(readFeatures, features_dir="/content/cough_valid_features/")

In [ ]:
!rm -rf /content/no_cough_data_valid
!mkdir -p /content/no_cough_data_valid
for index, row in negative_df_valid.iterrows():
    src = row["audio_path"]
    dst = "/content/no_cough_data_valid/"+row["uuid"]+".wav"
    copyfile(src, dst)
negative_df_valid['audio_path'] = negative_df_valid['audio_path'].apply(lambda x: "/content/no_cough_data_valid/"+os.path.basename(x))

In [ ]:
!rm -rf /content/no_cough_valid_features
!mkdir -p /content/no_cough_valid_features
%cd /content/kambria-challenge/generateMFCC/
!java -cp lib/jlibrosa-1.1.8-SNAPSHOT-jar-with-dependencies.jar:bin com.example.Main /content/no_cough_data_valid /content/no_cough_valid_features 16000 120

In [ ]:
negative_df_valid["feature"] = negative_df_valid["uuid"].apply(readFeatures, features_dir="/content/no_cough_valid_features/")

In [ ]:
final_aug_data_valid = pd.concat([positive_df_valid, negative_df_valid])
final_aug_data_valid

In [ ]:
!rm -rf /content/cough_data_test
!mkdir -p /content/cough_data_test
for index, row in positive_df_test.iterrows():
    src = row["audio_path"]
    dst = "/content/cough_data_test/"+row["uuid"]+".wav"
    copyfile(src, dst)
positive_df_test['audio_path'] = positive_df_test['audio_path'].apply(lambda x: "/content/cough_data_test/"+os.path.basename(x))

!rm -rf /content/cough_test_features
!mkdir -p /content/cough_test_features
%cd /content/kambria-challenge/generateMFCC/
!java -cp lib/jlibrosa-1.1.8-SNAPSHOT-jar-with-dependencies.jar:bin com.example.Main /content/cough_data_test /content/cough_test_features 16000 120

positive_df_test["feature"] = positive_df_test["uuid"].apply(readFeatures, features_dir="/content/cough_test_features/")

!rm -rf /content/no_cough_data_test
!mkdir -p /content/no_cough_data_test
for index, row in negative_df_test.iterrows():
    src = row["audio_path"]
    dst = "/content/no_cough_data_test/"+row["uuid"]+".wav"
    copyfile(src, dst)
negative_df_test['audio_path'] = negative_df_test['audio_path'].apply(lambda x: "/content/no_cough_data_test/"+os.path.basename(x))

!rm -rf /content/no_cough_test_features
!mkdir -p /content/no_cough_test_features
%cd /content/kambria-challenge/generateMFCC/
!java -cp lib/jlibrosa-1.1.8-SNAPSHOT-jar-with-dependencies.jar:bin com.example.Main /content/no_cough_data_test /content/no_cough_test_features 16000 120

negative_df_test["feature"] = negative_df_test["uuid"].apply(readFeatures, features_dir="/content/no_cough_test_features/")

final_aug_data_test = pd.concat([positive_df_test, negative_df_test])
final_aug_data_test

### Training

In [ ]:
def create_cnn():
    model = Sequential()
    model.add(Conv2D(16, (7,7), input_shape=(num_rows, num_columns, num_channels), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3,3), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (1,1), activation='relu', padding="same"))
    model.add(BatchNormalization())
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [ ]:
class DataGenerator(Sequence):
    def __init__(self,
                _X,
                batch_size=32,
                n_channels=1,
                n_columns=470,
                n_rows=120,
                shuffle=True):
        self.batch_size = batch_size
        self.X = _X
        self.n_channels = n_channels
        self.n_columns = n_columns
        self.n_rows = n_rows
        self.shuffle = shuffle
        self.img_indexes = np.arange(len(self.X))
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.img_indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temps = [self.img_indexes[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temps)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temps):
        X = np.empty((self.batch_size, self.n_rows, self.n_columns))
        y = np.empty((self.batch_size), dtype=int)
        for i, ID in enumerate(list_IDs_temps):
            x_features = self.X.iloc[ID]["feature"]
            label = self.X.iloc[ID]["assessment_result"]
            X[i] = x_features
            y[i] = label
        X = X.reshape(X.shape[0], self.n_rows, self.n_columns, self.n_channels)
        return X, y

In [ ]:
params = dict(
    batch_size=16,
    n_rows=num_rows,
    n_columns=num_columns,
    n_channels=num_channels,
)
params_train = dict(
    shuffle=True,
    **params
)
params_valid = dict(
    shuffle=False,
    **params
)

In [ ]:
import matplotlib.pyplot as plt

def plot_his(history):
    plt.figure(1, figsize = (15,8))
    plt.subplot(221)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'])
    plt.subplot(222)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'])
    plt.show()

In [ ]:
checkpoint_model_path = "/content/cough_detection.hdf5"

def train_model(model, train_gen, val_gen, fold):    
    metric = "val_loss"
    print("METRIC:", metric)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=["accuracy"])

    num_epochs = 100

    checkpointer = ModelCheckpoint(
        filepath=checkpoint_model_path,
        monitor=metric, verbose=1, save_best_only=True)
    es_callback = EarlyStopping(monitor=metric, patience=10, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor=metric, factor=0.3, patience=1, verbose=1, min_delta=0.0001, cooldown=1, min_lr=0.00001)

    history = model.fit(
        train_gen,
        epochs=num_epochs,
        validation_data=val_gen,
        callbacks=[checkpointer,es_callback,reduce_lr],
        verbose=1
    )

    plot_his(history)

In [ ]:
X_valid = final_aug_data_valid
X_train = final_aug_data
train_generator = DataGenerator(X_train, **params_train)
valid_generator = DataGenerator(X_valid, **params_valid)
cnn_model = create_cnn()
train_model(cnn_model, train_generator, valid_generator)

### Infer

In [ ]:
infer_df = final_aug_data_test
infer_df

In [ ]:
infer_df["assessment_result"].value_counts()

In [ ]:
params = dict(
    batch_size=1,
    n_rows=num_rows,
    n_columns=num_columns,
    n_channels=num_channels,
)
params_valid = dict(
    shuffle=False,
    **params
)
full_gen = DataGenerator(infer_df, **params_valid)

pred_models = []
cnn_model = create_cnn("mobilenet")
cnn_model.load_weights(checkpoint_model_path)
y_preds = cnn_model.predict(full_gen)
predictions = [p[0] for p in y_preds]
pred_models.append(predictions)

In [ ]:
infer_df["pred"] = np.average(pred_models, axis=0)
from sklearn.metrics import classification_report
print(classification_report(infer_df["assessment_result"], np.round(infer_df["pred"]), target_names=["not_cough", "cough"]))